# Word2Vec

In [ ]:
import gensim
gensim.__version__

'4.3.1'

In [ ]:
!pip install nltk==3.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 영어 데이터 다운로드 및 전처리

In [ ]:
import re
from lxml import etree
import urllib.request
import zipfile
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Word2Vec을 학습하기 위해서 데이터를 다운로드합니다.

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f2e215d59a0>)

In [ ]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

현재 영어 텍스트가 content_text에 저장되어져 있습니다. 이에 대해서 NLTK의 sent_tokenize를 통해서 문장을 구분해봅시다.

In [ ]:
len(content_text)

24062319

In [ ]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [ ]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

총 문장의 개수는 273,424개입니다.

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [ ]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


result => 이중 리스트

## 영어 Word2Vec 훈련시키기

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

여기서 Word2Vec의 하이퍼파라미터값은 다음과 같습니다.  

vector_size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.  
window = 컨텍스트 윈도우 크기  
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
workers = 학습을 위한 프로세스 수  
sg = 0은 CBOW, 1은 Skip-gram.  

In [ ]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8670949339866638), ('guy', 0.8084812760353088), ('lady', 0.7827594876289368), ('boy', 0.7626639604568481), ('gentleman', 0.7430579662322998), ('girl', 0.738659679889679), ('soldier', 0.7362387180328369), ('kid', 0.6873032450675964), ('son', 0.6749854683876038), ('poet', 0.6583216786384583)]


In [ ]:
model.wv["man"]

array([ 0.36962926, -2.2756646 ,  0.6699538 , -0.2637679 ,  1.7650979 ,
        0.41376078,  0.6191673 ,  0.31161126, -1.0972432 ,  0.08376483,
       -1.5669507 , -1.3366003 , -1.0022043 ,  0.7188004 , -0.38247004,
       -0.17858262,  0.82908124, -0.20770907,  0.7418748 , -0.6679165 ,
        0.4786106 ,  0.42078748,  0.08435857, -0.7552861 ,  0.714697  ,
       -0.34108162, -1.7101638 , -0.78365374,  0.05432801, -0.943095  ,
       -1.0635644 , -0.14630418,  0.89336675,  0.8741964 , -0.2220911 ,
       -0.8707844 , -0.96160686, -0.8662834 , -1.0567826 , -0.42119226,
        1.4371676 , -2.357533  , -0.93037105,  1.2121828 , -0.04018362,
       -0.41777614, -1.7571156 , -1.4297916 , -0.7510524 ,  0.4420943 ,
       -0.5157603 , -3.1381721 , -0.48090369,  1.314909  ,  0.08452661,
       -0.7191689 , -0.7336814 , -0.6488322 , -1.6281276 ,  0.29343018,
       -0.77411216,  0.5295586 ,  0.7218926 ,  0.55249   , -2.193133  ,
        0.85211885, -0.0398171 , -0.22079284, -0.33323535,  1.23

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [ ]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8670949339866638), ('guy', 0.8084812760353088), ('lady', 0.7827594876289368), ('boy', 0.7626639604568481), ('gentleman', 0.7430579662322998), ('girl', 0.738659679889679), ('soldier', 0.7362387180328369), ('kid', 0.6873032450675964), ('son', 0.6749854683876038), ('poet', 0.6583216786384583)]


In [ ]:
# 현재 경로
%pwd

'/content'

## 한국어 데이터 다운로드 및 전처리

KoNLPy의 OKT 등은 형태소 분석 속도가 너무 느립니다. 그래서 Mecab을 설치하겠습니다.  
단, Mecab은 형태소 분석 속도는 빠르지만 설치하는데 시간이 좀 걸립니다.

In [ ]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 47.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.5/581.5 KB 40.3 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1251 sha256=c0f5e6a5860a157df8448e62800382b5136ada2c44c884dbacd5d59ce831ca92
  Stored in directory: /root/.cache/pip/wheels/63/09/15/cc401a7f8d041043978f3f60e64f7d65014522e104b7c9d1f2
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0     

In [ ]:
import urllib.request
from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f2e46c39a00>)

In [ ]:
train_data = pd.read_table('ratings.txt')

In [ ]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
print(len(train_data)) # 리뷰 개수 출력

200000


In [ ]:
# NULL 값 존재 유무
print(train_data.isnull().values.any())

True


In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [ ]:
print(len(train_data)) # 리뷰 개수 출력

199992


In [ ]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

<ipython-input-27-d10eedfa8951>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [ ]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1


In [ ]:
# 불용어 정의
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [ ]:
# 형태소 분석기 mecab을 사용한 토큰화 작업 (다소 시간 소요)
mecab = Mecab()
tokenized_data = []
for sentence in train_data['document']:
    temp_X = mecab.morphs(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [ ]:
print(tokenized_data[:3])

[['어릴', '때', '보', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ'], ['디자인', '배우', '학생', '으로', '외국', '디자이너', '그', '일군', '전통', '통해', '발전', '해', '문화', '산업', '부러웠', '는데', '사실', '우리', '나라', '에서', '그', '어려운', '시절', '끝', '까지', '열정', '지킨', '노라노', '같', '전통', '있', '어', '저', '같', '사람', '꿈', '꾸', '이뤄나갈', '수', '있', '다는', '것', '감사', '합니다'], ['폴리스', '스토리', '시리즈', '부터', '뉴', '까지', '버릴', '께', '하나', '없', '음', '최고']]


## 한국어 Word2Vec 훈련시키기

nested list를 데이터로 넣어주셔야 합니다.

[['나는', '사과를', 먹는다'], ['이', '영화', '는', '재밌어']]

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

(18134, 100)

In [ ]:
print(model.wv.most_similar("최민식"))

[('한석규', 0.8591922521591187), ('송강호', 0.8543269634246826), ('설경구', 0.8538635969161987), ('안성기', 0.8446723222732544), ('드니로', 0.8391854166984558), ('워싱턴', 0.8360030651092529), ('백윤식', 0.8285912871360779), ('김주혁', 0.8282619118690491), ('이미숙', 0.8279692530632019), ('김희선', 0.8269199728965759)]


In [ ]:
model.wv['최민식']

array([ 0.02108885,  0.08945522, -0.12406699, -0.01871551, -0.16449685,
       -0.128411  ,  0.3481762 ,  0.24950962,  0.045748  ,  0.00277088,
        0.16452602, -0.44035956, -0.16201323,  0.18551268, -0.07007945,
        0.03434223,  0.0064904 , -0.05555565,  0.39582577, -0.52943975,
        0.42246124, -0.19480792,  0.1801543 ,  0.34763893, -0.00181176,
       -0.01084024, -0.00613135, -0.2733991 , -0.00622511,  0.184433  ,
        0.30129734,  0.00990239,  0.3493642 ,  0.02089984, -0.1409225 ,
        0.38785252,  0.14001994,  0.23662788, -0.19740976, -0.5040234 ,
        0.07080445, -0.45596215,  0.16358747,  0.13696729,  0.23678483,
       -0.0963066 , -0.1014014 , -0.24432757,  0.14955874, -0.20201832,
        0.01346016,  0.32508394,  0.04137043, -0.05903203,  0.0023563 ,
       -0.22329955, -0.02075332,  0.03136083,  0.11203606, -0.0677736 ,
        0.24316053,  0.32754412, -0.42695975, -0.01665941, -0.29452705,
        0.0670014 ,  0.07910407,  0.14039207, -0.53783804,  0.41

In [ ]:
print(model.wv.most_similar("히어로"))

[('호러', 0.8413481116294861), ('슬래셔', 0.8230418562889099), ('하이틴', 0.7898564338684082), ('정통', 0.7874332070350647), ('최고봉', 0.7855408787727356), ('패러디', 0.782534658908844), ('고어', 0.7777903079986572), ('로코', 0.7744789123535156), ('무비', 0.7713674902915955), ('블록버스터', 0.7706264853477478)]


In [ ]:
# 영어 모델이 저장된 경로로 이동
%cd /content

/content


In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('kor_w2v') # 모델 저장

# FastText

## Word2Vec의 OOV 문제 확인해보기

OOV 문제(Out-Of-Vocabulary Problem) : Vocabulary에 존재하지 않는 단어가 등장하는 문제

In [ ]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # Word2Vec 모델 로드

In [ ]:
model_result = loaded_model.most_similar("overacting")
print(model_result)

KeyError: ignored

In [ ]:
model_result = loaded_model.most_similar("memory")
print(model_result)

[('brain', 0.6911921501159668), ('imagination', 0.6807860136032104), ('vision', 0.6735898852348328), ('consciousness', 0.6577538847923279), ('perception', 0.6522263884544373), ('gut', 0.6279733180999756), ('mind', 0.6269828081130981), ('body', 0.6263871192932129), ('genome', 0.620076596736908), ('cognitive', 0.6118467450141907)]


In [ ]:
model_result = loaded_model.most_similar("memorry")
print(model_result)

KeyError: ignored

In [ ]:
model_result = loaded_model.most_similar("electrofishing")
print(model_result)

KeyError: ignored